In [1]:
from hdfs import InsecureClient
from pyspark.sql import SparkSession
import json
from elasticsearch import Elasticsearch,helpers

In [2]:
es = Elasticsearch(hosts="elasticsearch:9200",http_auth=("elastic","elastic@1234"))

In [49]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar --driver-class-path /home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-spark-20_2.11-7.11.1.jar --driver-class-path /home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-spark-20_2.11-7.11.1.jar pyspark-shell'

In [3]:
client = InsecureClient("http://hadoop-filebeat:50070",user="nifi")

In [4]:
client.list('/dfs-data')

['00cf8c06-8263-4748-8b6b-421bdd141553',
 '03f31c9a-8e29-4bc5-a0ee-240d3aa42d87',
 '0ed14d9b-517b-48e5-a21f-bc48069c6859',
 '13a0978f-00b1-4b1f-be1a-78d039b10880',
 '1beaa5a0-c44e-4edf-907f-8db6bff6e238',
 '1c70e0d6-aa67-40b8-bd87-de349efa863e',
 '34ffde6a-4e2e-4c37-8557-12fc90c6164f',
 '379fa46f-d401-4b09-8ca0-f28b38743bec',
 '8251ce38-7b37-4daa-88b4-3ba34bda39a4',
 '9629c030-352e-4dd1-936e-49b5cafb4949',
 'cb2dbc2b-e2d7-4548-abf5-8c461ea67d3e',
 'd560be0f-1c0c-462b-ac1b-a8516ec7fe7c',
 'e7b39a25-5c87-44b5-ab92-93bba6498bec',
 'eb0af216-1dcd-4c15-8173-cdaf1a3c587f',
 'fd2413a2-6d6c-4b2d-ad2d-fa7a965fda56']

In [5]:
spark = (SparkSession
        .builder
        .appName("hdfsprocessor")
        .config("spark.jars","/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar")
        .config("spark.jars","/home/jovyan/spark_conf/elasticsearch-hadoop-7.11.1/dist/elasticsearch-spark-20_2.11-7.11.1.jar") 
        .config("spark.es.nodes","elasticsearch")
        .config("spark.es.port","9200")
        .getOrCreate())

In [55]:
file = "1e20e01b-1c63-4de4-873d-0d6e0cfec962"
df_load = spark.read.format("csv").option("delimiter", ",").option("header", "false").load(f"hdfs://hadoop-filebeat:8020/dfs-data/{file}")
final_df = df_load.select(df_load._c0.alias('collision_id'), df_load._c1.alias('crash_date'), df_load._c2.alias('crash_time'), df_load._c3.alias('borough'), df_load._c4.alias('zip_code'), df_load._c5.alias('latitude'), df_load._c6.alias('longitude'), df_load._c7.alias('on_street_name'), df_load._c9.alias('number_of_persons_injured'), df_load._c10.alias('number_of_persons_killed'), df_load._c19.alias('vehicle_type'))
# final_df.coalesce(1).write.format("csv").save(f"http://filebeat:5044/{file}.csv")

In [22]:
# !pyspark --jars elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar --driver-class-path elasticsearch-hadoop-7.11.1/dist/elasticsearch-hadoop-7.11.1.jar

In [56]:
df = spark.read.csv(f"hdfs://hadoop-filebeat:8020/dfs-data/{file}",header=False)

In [78]:
df_final = df.select(df._c0.alias('collision_id'), df._c1.alias('crash_date'), df._c2.alias('crash_time'), df._c3.alias('borough'), df._c4.alias('zip_code'), df._c5.alias('latitude'), df._c6.alias('longitude'), df._c7.alias('on_street_name'), df._c9.alias('number_of_persons_injured'), df._c10.alias('number_of_persons_killed'), df._c19.alias('vehicle_type'))

In [79]:
# rdd = df_final.toJSON().map(lambda j: json.loads(j)).collect()
rdd_mapped = df_final.rdd.map(lambda x:x.asDict())

In [80]:
final_rdd = rdd_mapped.map(transform)

In [10]:
final_json_list = []

In [11]:
for file in client.list('/dfs-data'):
    df_load = spark.read.format("csv").option("delimiter", ",").option("header", "false").load(f"hdfs://hadoop-filebeat:8020/dfs-data/{file}")
    final_df = df_load.select(df_load._c0.alias('collision_id'), df_load._c1.alias('crash_date'), df_load._c2.alias('crash_time'), df_load._c3.alias('borough'), df_load._c4.alias('zip_code'), df_load._c5.alias(
    'latitude'), df_load._c6.alias('longitude'), df_load._c7.alias('on_street_name'), df_load._c9.alias('number_of_persons_injured'), df_load._c10.alias('number_of_persons_killed'), df_load._c19.alias('vehicle_type'))
#     final_df.coalesce(1).write.format("csv").save(f"hdfs://hadoop-filebeat:8020/csv-data/{file}.csv")
    json_ = final_df.toJSON().map(lambda j: json.loads(j)).collect()
    final_json_list.extend(json_)

In [13]:
helpers.bulk(
        es,
        final_json_list,
        index='elk_project',
        doc_type='_doc')

/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


(15, [])